In [6]:
! . ../honor/download_cornell.sh

--2021-06-15 17:16:30--  https://github.com/Conchylicultor/DeepQA/raw/master/data/cornell/movie_conversations.txt
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Conchylicultor/DeepQA/master/data/cornell/movie_conversations.txt [following]
--2021-06-15 17:16:31--  https://raw.githubusercontent.com/Conchylicultor/DeepQA/master/data/cornell/movie_conversations.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6760930 (6.4M) [text/plain]
Saving to: 'movie_conversations.txt'

movie_conversations 100%[===================>]   6.45M  5.64MB/s    in 1.1s    

2021-06-15 17:16:32 (5.64 MB/s) - 'movie_c

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import time
tf.__version__

'1.4.0'

In [8]:
lines = open('data/cornell/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('data/cornell/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

The sentences that we will be using to train our model.

In [9]:
lines[:10]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?']

The sentences' ids, which will be processed to become our input and target data.

In [10]:
conv_lines[:10]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

In [11]:
id2line = {}
for line in lines:
    line_ = line.split(" +++$+++ ")
    if len(line_) == 5:
        id2line[line_[0]] = line_[4]

In [12]:
convs = []
for line in conv_lines:
    _line = line.split(" +++$+++ ")[-1][1:-1].replace("'", "").replace(" ", "")
    convs.append(_line.split(','))

In [13]:
convs[:10]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366']]

dialog example

In [14]:
ID = 1488
for i in convs[ID]:
    print(id2line[i])

We've blown the computer!  Elaine!  Set course change!
Set!
Now!
Compute!


Sort the sentences into questions (inputs) and answers (targets)

In [15]:
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])

Check if we have loaded the data correctly

In [17]:
limit = 0
for i in range(limit, limit+5):
    print(questions[i])
    print(answers[i])
    print()

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.

Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.

Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

You're asking me out.  That's so cute. What's your name again?
Forget it.

No, no, it's my fault -- we didn't have a proper introduction ---
Cameron.



Compare lengths of questions and answers

In [18]:
print(len(questions))
print(len(answers))

221616
221616


In [19]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [20]:
clean_questions = [clean_text(line) for line in questions]
clean_answers = [clean_text(line) for line in answers]

Take a look at some of the data to ensure that it has been cleaned well.

In [21]:
limit = 0
for i in range(limit, limit+5):
    print(clean_questions[i])
    print(clean_answers[i])
    print()

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
well i thought we would start with pronunciation if that is okay with you

well i thought we would start with pronunciation if that is okay with you
not the hacking and gagging and spitting part  please

not the hacking and gagging and spitting part  please
okay then how about we try out some french cuisine  saturday  night

you are asking me out  that is so cute that is your name again
forget it

no no it is my fault  we did not have a proper introduction 
cameron



In [22]:
lengths = [len(line.split()) for line in clean_questions]

In [23]:
lengths = pd.DataFrame(lengths, columns=['counts'])
lengths.describe()

,counts
count,221616.000000
mean,10.684120
std,11.804262
min,0.000000
25%,4.000000
50%,7.000000
75%,13.000000
max,319.000000


In [24]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

16.0
19.0
23.0
32.0
57.0


Remove questions and answers that are shorter than 2 words and longer than 32 words.

In [26]:
min_line_length = 2
max_line_length = 32

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

for question, answers in zip(clean_questions, clean_answers):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length and \
        len(answers.split()) >= min_line_length and len(answers.split()) <= max_line_length:
        short_questions.append(question)
        short_answers.append(answers)

print("clean_answers len = ", len(clean_answers))
print("short_answers len = ", len(short_answers))

clean_answers len =  221616
short_answers len =  168618


Compare the number of lines we will use with the total number of lines.

In [27]:
print("# of questions:", len(short_questions))
print("# of answers:", len(short_answers))
print("% of data used: {}%".format(round(len(short_questions)/len(questions),4)*100))

# of questions: 168618
# of answers: 168618
% of data used: 76.09%


Create a dictionary for the frequency of the vocabulary

In [28]:
vocab = {}
for question in short_questions:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for answer in short_answers:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
print(len(vocab))

55594


Remove rare words from the vocabulary.

In [29]:
threshold = 7
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 55594
Size of vocab we will use: 13598


In case we want to use a different vocabulary sizes for the source and target text, we can set different threshold values. Nonetheless, we will create dictionaries to provide a unique integer for each word.

In [30]:

questions_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        questions_vocab_to_int[word] = word_num
        word_num += 1
        
answers_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        answers_vocab_to_int[word] = word_num
        word_num += 1

Add the unique tokens to the vocabulary dictionaries.

In [31]:
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int)+1
    
for code in codes:
    answers_vocab_to_int[code] = len(answers_vocab_to_int)+1

Create dictionaries to map the unique integers to their respective words. i.e. an inverse dictionary for vocab_to_int.

In [32]:
questions_int_to_vocab = {v_i: v for v, v_i in questions_vocab_to_int.items()}
answers_int_to_vocab = {v_i: v for v, v_i in answers_vocab_to_int.items()}

Check the length of the dictionaries.

In [33]:
print(len(questions_vocab_to_int))
print(len(questions_int_to_vocab))
print(len(answers_vocab_to_int))
print(len(answers_int_to_vocab))

13602
13602
13602
13602


Add the end of sentence token to the end of every answer.

In [34]:
for i in range(len(short_answers)):
    short_answers[i] += ' <EOS>'

Convert the text to integers. Replace any words that are not in the respective vocabulary with "UNK"

In [35]:
questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        ints.append(questions_vocab_to_int[word] if word in questions_vocab_to_int else questions_vocab_to_int['<UNK>'])
    questions_int.append(ints)

answers_int = []
for question in short_answers:
    ints = []
    for word in question.split():
        ints.append(answers_vocab_to_int[word] if word in answers_vocab_to_int else answers_vocab_to_int['<UNK>'])
    answers_int.append(ints)

Check the lengths

In [36]:
print(len(questions_int))
print(len(answers_int))

168618
168618


Calculate what percentage of all words have been replaced with UNK

In [37]:
word_count = 0
unk_count = 0

for question in questions_int:
    for word in question:
        if word == questions_vocab_to_int['<UNK>']:
            unk_count += 1
        word_count += 1
    
for answer in answers_int:
    for word in answer:
        if word == answers_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
unk_ratio = round(unk_count/word_count,4)*100
    
print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))

Total number of words: 3404792
Number of times <UNK> is used: 89685
Percent of words that are <UNK>: 2.63%


 Sort questions and answers by the length of questions.  
 This will reduce the amount of padding during training  
 Which should speed up training and help to reduce the loss

In [38]:
sorted_questions = []
sorted_answers = []

for length in range(1, max_line_length+1):
    for i in enumerate(questions_int):
        if len(i[1]) == length:
            sorted_questions.append(questions_int[i[0]])
            sorted_answers.append(answers_int[i[0]])

print(len(sorted_questions))
print(len(sorted_answers))
print()
for i in range(3):
    print(sorted_questions[i])
    print(sorted_answers[i])
    print()

168618
168618

[733, 7329]
[12941, 9162, 9162, 9162, 2880, 6665, 6064, 12941, 3274, 11338, 6030, 2229, 1726, 8076, 13600]

[12941, 2963]
[5051, 7140, 171, 8897, 6665, 6043, 6114, 13601, 5950, 5051, 13600]

[10417, 13243]
[12551, 3971, 8753, 1147, 5051, 6634, 8897, 9945, 13600]



### NN

In [44]:
class AttentionModel(object):
    pass

In [47]:
def model_inputs(self):
    '''Create palceholders for inputs to the model'''
    self.input_data = tf.placeholder(tf.int32, [None, None], name='input')
    self.targets = tf.placeholder(tf.int32, [None, None], name='targets')
    self.lr = tf.placeholder(tf.float32, name='learning_rate')
    # Placeholder for a dropout keep probability.
    self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [51]:
AttentionModel.__model_inputs = classmethod(model_inputs)

In [48]:
def process_encoding_input(self, target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    self.dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

In [52]:
AttentionModel.__process_encoding_input = classmethod(process_encoding_input)

In [54]:
tf.strided_slice?

In [49]:
def encoding_layer(self, rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    '''Create the encoding layer'''
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=self.keep_prob)
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop]*num_layers)
    _, self.enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw=enc_cell,
                                                   cell_bw=enc_cell,
                                                   sequence_length=sequence_length,
                                                   inputs=rnn_inputs, 
                                                   dtype=tf.float32)

In [53]:
AttentionModel.__encoding_layer = classmethod(encoding_layer)

In [ ]:
def decoding_layer_train(self, encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob, batch_size, rnn_size=20):
    '''Decode the training data'''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(dec_cell.output_size)
    
    #att_keys, att_vals, att_score_fn, att_construct_fn = \
    attn_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell, attention_mechanism)
    
    
    ##################################################################################
    train_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                     att_keys,
                                                                     att_vals,
                                                                     att_score_fn,
                                                                     att_construct_fn,
                                                                     name="attn_dec_train")
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                              train_decoder_fn, 
                                                              dec_embed_input, 
                                                              sequence_length, 
                                                              scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, self.keep_prob)
    self.output = output_fn(train_pred_drop)
    ##################################################################################    
    
    
    out_cell = tf.contrib.rnn.OutputProjectionWrapper(
                attn_cell, vocab_size, reuse)
    decoder = tf.contrib.seq2seq.BasicDecoder(
                cell=out_cell, helper=helper,
                initial_state=out_cell.zero_state(
                    dtype=tf.float32, batch_size=batch_size))
                #initial_state=encoder_final_state)
    outputs = tf.contrib.seq2seq.dynamic_decode(
                decoder=decoder, output_time_major=False,
                impute_finished=True, maximum_iterations=output_max_length
            )
            return outputs[0]

In [82]:
tf.contrib.rnn.OutputProjectionWrapper?

In [65]:
tf.contrib.rnn.BasicLSTMCell?

In [68]:
tf.contrib.seq2seq.AttentionWrapper?

In [69]:
tf.contrib.seq2seq.dynamic_rnn_decoder?

Object `tf.contrib.seq2seq.dynamic_rnn_decoder` not found.


In [70]:
tf.contrib.seq2seq.prepare_attention?

Object `tf.contrib.seq2seq.prepare_attention` not found.


In [72]:
tf.contrib.layers.fully_connected?

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob, batch_size):
    '''Decode the training data'''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
            tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    train_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                     att_keys,
                                                                     att_vals,
                                                                     att_score_fn,
                                                                     att_construct_fn,
                                                                     name = "attn_dec_train")
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                              train_decoder_fn, 
                                                              dec_embed_input, 
                                                              sequence_length, 
                                                              scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)
    return output_fn(train_pred_drop)



In [ ]:
def decoding_layer(self, dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, vocab_to_int, keep_prob, batch_size):
    '''Create the decoding cell and input the parameters for the training and inference decoding layers'''
    
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=self.keep_prob)
        dec_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
        
        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, 
                                                                vocab_size, 
                                                                None, 
                                                                scope=decoding_scope,
                                                                weights_initializer = weights,
                                                                biases_initializer = biases)
    ##################################################################################  
        self.train_logits = decoding_layer_train(encoder_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            sequence_length, 
                                            decoding_scope, 
                                            output_fn, 
                                            keep_prob, 
                                            batch_size)
        decoding_scope.reuse_variables()
        self.infer_logits = decoding_layer_infer(encoder_state, 
                                            dec_cell, 
                                            dec_embeddings, 
                                            vocab_to_int['<GO>'],
                                            vocab_to_int['<EOS>'], 
                                            sequence_length - 1, 
                                            vocab_size,
                                            decoding_scope, 
                                            output_fn, keep_prob, 
                                            batch_size)      
    ##################################################################################          
        
    train_helper = tf.contrib.seq2seq.TrainingHelper(output_embed, output_lengths)
    pred_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        embeddings, start_tokens=tf.to_int32(start_tokens), end_token=1)

In [81]:
tf.variable_scope?